# Implementing Boyer-Moore Algorithm

In [11]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z

def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp

def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [12]:
# GCTAGCTCTACGAGTCTA
p = 'TCAA'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.bad_character_rule(2, 'T')

2

In [13]:
# GCTAGCTCTACGAGTCTA
# ACTA
p = 'ACTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.good_suffix_rule(0)

3

In [14]:
# ACACGCTCTACGAGTCTA
# ACAC
p = 'ACAC'
p_bm = BoyerMoore(p, alphabet='ACGT')
p_bm.match_skip()

2

In [15]:
def boyer_moore(p, p_bm, t):   #p_bm = preprocessing
    i = 0
    occurances = []
    while i < len(t) - len(p) + 1 :
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):  #from last to zero scan
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurances.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurances

In [16]:
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = BoyerMoore(p, alphabet='ACGT')   # p_bm = p er preprocessed

In [17]:
boyer_moore(p, p_bm, t)   #occurances print korbe

[6, 14]

# Implementing a k-mer index

In [18]:
import bisect

class Index(object):
    def __init__(self, t, k):
        self.k = k                                      # k-mer length = k
        self.index = []   
        for i in range(len(t) - k + 1):
            self.index.append((t[i:i+k], i))            # kmer gula bosano hoise (string, index) touple diye
        self.index.sort()
        
    def query(self, p):
        kmer = p[:self.k]                                # given str p theke k ta char nite hbe kmer hisabe
        i = bisect.bisect_left(self.index, (kmer, -1))   # i = kmer kon index e boste parbe seta find korbe (syxtax e amon), right the left e khujbe and leftmost index return krbe
        hits = []
        while i < len(self.index):
            if self.index[i][0] != kmer:                 # i pawar por i jodi kmer na hoy break 
                break
            hits.append(self.index[i][1])                # i jodi kmer hoy tahole sei kmer er index collect kora hbe
            i += 1                                       # i leftmost index return krbe, tai i++ kore dekhte hbe aro hit ache kina
        return hits

In [19]:
def queryIndex(p, t, index):
    k = index.k                       # index = Index class er obj. so k = joto k set kora hoise
    offsets = []                      # kothay kothay full p str match korse
    for i in index.query(p):          # jekhane jekhane k-mer pabe oikhan theke purata scan kore nibe
        if p[k:] == t[i+k:i+len(p)]:  # p[k:] karon first k ta k-mer diyei hoye gese. tai k theke len(p) porjonto check krte hbe
            offsets.append(i) 
    return offsets

In [20]:
t = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
p = 'GGTATTCGGGA'

In [21]:
index = Index(t, 4)              # t str diye 4-mer set hbe
print(queryIndex(p, t, index))   # query diye only k-mer search kore, queryIndex diye full p search kore

[21, 68]


# Approximate Match Implementation

In [1]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z

def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp

def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [2]:
def boyer_moore(p, p_bm, t):   #p_bm = preprocessing
    i = 0
    occurances = []
    while i < len(t) - len(p) + 1 :
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):  #from last to zero scan
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurances.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurances

In [3]:
def approximate_match(p, t, n):                           # n = max num of mismatches
    
    all_matches = set()
    segment_length = round(len(p)/(n+1))                  # n ta mismatch allowed. tai (n+1) ta tukra korte hbe p ke.
    for i in range(n+1):
        start = i*segment_length                          # (jototomo tukra * tukrar length) theke start. like 0*3=0 or 1*3=3       
        end = min((i+1)*segment_length, len(p))           # (i+1)*segment_len hobe end point
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')  # preprocess korte pathate hbe
        matches = boyer_moore(p[start:end], p_bm, t)      # koyta match holo tader offset index dibe
        
        for m in matches:                                 # proti ta match e jabo (segment match index)
            if m < start or m-start+len(p) > len(t):      # seg jekhane start hoise, tar age toh match howa possible na, ar match koranor somoy len(t) exceed kora jabena
                continue
                
            mismatches = 0
            for j in range(0, start):                     # segment ta p er jekhan theke start hoise tar ag porjonto p==t hoise kina check
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            
            for j in range(end, len(p)):                  # segment jekhane ses hoise tar por theke p er ses porjonto p==t kina check
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            
            if mismatches <= n:
                all_matches.add(m-start)                  # expected result set e add kore fela
    return list(all_matches)
    

In [4]:
p = 'AACTTG'
t = 'CACTTAATTTG'
print(approximate_match(p, t, 2))

[0, 5]


In [5]:
print(t[5:])

AATTTG


In [6]:
print(approximate_match(p, t, 1))

[5]


In [9]:
print(approximate_match(p, t, 2)) # 2 er besi dile error khacche

[0, 5]


In [10]:
print(approximate_match(p, t, 0))

[]


# Quiz Week 2

In [15]:
import urllib.request

url = 'http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta'
filename = 'chr1.GRCh38.excerpt.fasta'
urllib.request.urlretrieve(url, filename)
print(filename)

chr1.GRCh38.excerpt.fasta


In [1]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [2]:
def naive(p, t):
    scan = 0
    occurances = []
    for i in range(len(t)-len(p)+1):
        scan += 1
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match == True:
            occurances.append(i)
    return scan

In [3]:
scan = naive('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG', genome)
print(scan)

799954


In [4]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z

def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp

def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [5]:
def boyer_moore(p, p_bm, t):   #p_bm = preprocessing
    i = 0
    occurances = []
    scan = 0
    while i < len(t) - len(p) + 1 :
        scan += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):  #from last to zero scan
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurances.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return scan

In [6]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(p, alphabet='ACGT')
print(boyer_moore(p, p_bm, t))

127974


In [7]:
def boyer_moore(p, p_bm, t):   #p_bm = preprocessing
    i = 0
    occurances = []
    while i < len(t) - len(p) + 1 :
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):  #from last to zero scan
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurances.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurances

In [8]:
def approximate_match(p, t, n):                           # n = max num of mismatches
    
    all_matches = set()
    segment_length = round(len(p)/(n+1))                  # n ta mismatch allowed. tai (n+1) ta tukra korte hbe p ke.
    for i in range(n+1):
        start = i*segment_length                          # (jototomo tukra * tukrar length) theke start. like 0*3=0 or 1*3=3       
        end = min((i+1)*segment_length, len(p))           # (i+1)*segment_len hobe end point
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')  # preprocess korte pathate hbe
        matches = boyer_moore(p[start:end], p_bm, t)      # koyta match holo tader offset index dibe
        
        for m in matches:                                 # proti ta match e jabo (segment match index)
            if m < start or m-start+len(p) > len(t):      # seg jekhane start hoise, tar age toh match howa possible na, ar match koranor somoy len(t) exceed kora jabena
                continue
                
            mismatches = 0
            for j in range(0, start):                     # segment ta p er jekhan theke start hoise tar ag porjonto p==t hoise kina check
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            
            for j in range(end, len(p)):                  # segment jekhane ses hoise tar por theke p er ses porjonto p==t kina check
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            
            if mismatches <= n:
                all_matches.add(m-start)   
    return list(all_matches)


In [9]:
def naive_2mm(p, t):
    occurances = []
    for i in range(len(t)-len(p)+1):
        match = True
        cnt=0
        for j in range(len(p)):
            if t[i+j] != p[j]:
                cnt+=1
        if cnt>2:
            match = False
        if match == True:
            occurances.append(i)
    return occurances

In [24]:
import bisect

class Index(object):
    def __init__(self, t, k):
        self.k = k                                      # k-mer length = k
        self.index = []   
        for i in range(len(t) - k + 1):
            self.index.append((t[i:i+k], i))            # kmer gula bosano hoise (string, index) touple diye
        self.index.sort()
        
    def query(self, p):
        for j in range(len(p)-self.k+1):
            kmer = p[:self.k]                                # given str p theke k ta char nite hbe kmer hisabe
            i = bisect.bisect_left(self.index, (kmer, -1))   # i = kmer kon index e boste parbe seta find korbe (syxtax e amon), right the left e khujbe and leftmost index return krbe
            hits = []
            while i < len(self.index):
                c = 0
                for pp in range(len(kmer)):
                    if self.index[i][0][pp] != kmer[pp]:
                        c+=1
                if c>2:
                    break
                #if len(approximate_match(self.index[i][0], p, 2))==0:
                    #break
                hits.append(self.index[i][1])                # i jodi kmer hoy tahole sei kmer er index collect kora hbe
                i += 1                                       # i leftmost index return krbe, tai i++ kore dekhte hbe aro hit ache kina
        return hits

In [25]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [26]:
index = Index(t, 8)              # t str diye 8-mer set hbe
print(index.query(p))  # query diye only k-mer search kore, queryIndex diye full p search kore
#'''print(p)
for i in index.query(p):
    print('p = ', p)
    print('s = ', genome[i:i+24])
    print(p==genome[i:i+24])


[56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061, 83871]
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCACGCCTGTAAT
True
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCAGGCGCCTGTAGT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGATTCATGCCTGTAAT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCATGCCTGTAAT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCATGCCTGTAAT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCACACCTGTAAT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGTTCACGCCTGTAAT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCACGCCTGTAAT
True
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCACGCCTGTAAT
True
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCACGCCTGTAAT
True
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCATGCCTGTAAT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCTCACGCCTGTAAT
True
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCGCGGTGGCAGGCGCCTGTAGT
False
p =  GGCGCGGTGGCTCACGCCTGTAAT
s =  GGCG

In [26]:
def queryIndex(p, t, index):
    k = index.k                       # index = Index class er obj. so k = joto k set kora hoise
    offsets = []                      # kothay kothay full p str match korse
    for i in index.query(p):  
        if p[k:] == t[i+k:i+len(p)]:  # p[k:] karon first k ta k-mer diyei hoye gese. tai k theke len(p) porjonto check krte hbe   
            offsets.append(i) 
    return offsets

In [27]:
print(queryIndex(p, t, index))

[56922, 262042, 364263, 657496, 717706, 551134, 273669]


In [49]:
genome[83871:83871+24]

'GGCGCGTGCCTGTAATCCCAGCTA'

In [50]:
len(genome)

800000

In [87]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            #if self.index[i][0] != subseq:
            nv = naive_2mm(self.index[i][0], subseq)
            if len(nv)==0:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [88]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [89]:
p = 'TTATAT'
print(ind.query(p[0:]))

[1]


In [90]:
print(ind.query(p[1:]))

[1]


In [91]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = genome

In [92]:
ind = SubseqIndex(genome, 8, 3)

In [93]:
genome[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [94]:
print(ind.index[:100])

[('AAAAAAAA', 1879), ('AAAAAAAA', 1882), ('AAAAAAAA', 1885), ('AAAAAAAA', 1888), ('AAAAAAAA', 2989), ('AAAAAAAA', 2992), ('AAAAAAAA', 2995), ('AAAAAAAA', 2996), ('AAAAAAAA', 2998), ('AAAAAAAA', 2999), ('AAAAAAAA', 3774), ('AAAAAAAA', 6370), ('AAAAAAAA', 7241), ('AAAAAAAA', 33796), ('AAAAAAAA', 33828), ('AAAAAAAA', 33831), ('AAAAAAAA', 35720), ('AAAAAAAA', 37621), ('AAAAAAAA', 38271), ('AAAAAAAA', 38910), ('AAAAAAAA', 38913), ('AAAAAAAA', 43081), ('AAAAAAAA', 43084), ('AAAAAAAA', 43087), ('AAAAAAAA', 43090), ('AAAAAAAA', 46338), ('AAAAAAAA', 46341), ('AAAAAAAA', 54148), ('AAAAAAAA', 54151), ('AAAAAAAA', 54154), ('AAAAAAAA', 54157), ('AAAAAAAA', 57205), ('AAAAAAAA', 57206), ('AAAAAAAA', 57207), ('AAAAAAAA', 57208), ('AAAAAAAA', 57209), ('AAAAAAAA', 57210), ('AAAAAAAA', 57211), ('AAAAAAAA', 57212), ('AAAAAAAA', 59231), ('AAAAAAAA', 59234), ('AAAAAAAA', 59237), ('AAAAAAAA', 59240), ('AAAAAAAA', 59243), ('AAAAAAAA', 60945), ('AAAAAAAA', 65524), ('AAAAAAAA', 69483), ('AAAAAAAA', 69486), ('AA

In [95]:
print(ind.query(p))

[56922, 67486, 83863, 84641, 84775, 124024, 147558, 191452, 199607, 262042, 262174, 273669, 322735, 364263, 421354, 454332, 465647, 471966, 472634, 489019, 558456, 579737, 596898, 635931, 651523, 657496, 658702, 681737, 707151, 712449, 717706, 719418, 719557, 746620, 747359, 171452, 251261, 261099, 278079, 292141, 442303, 460493, 499922, 613117, 775976, 215147, 222432, 441721, 511100]


In [96]:
print(len(ind.query(p)))

49


In [97]:
print(len(ind.query(p[0:])))

49


In [98]:
len(ind.query(p[1:]))

105

In [99]:
len(ind.query(p[2:]))

15

In [100]:
len(ind.query(p[3:]))

0

In [101]:
len(ind.query(p[4:]))

0

In [104]:
len(ind.query(p[7:]))

0